# Transformar arquivo SPSS para formato SPSS-SAV

SPSS é um software estatístico muito comum em alguns domínios como, por exemplo, Market Research.
O sofware possui uma linguagem de script própria onde é possivel ler um arquivo excel e transformá-lo em SAV (formato nativo do SPSS), porém é necessario um considerável esforço de programação.

O objetivo deste código é converter um arquivo Excel com o menor esforço de programação possivel. Para atingir este propósito usaremos um arquivo JSON que servirá de metadata.

Este arquivo metadata ajuda a definir:
* O nome das colunas no arquivo Excel
* O label de cada coluna
* O tipo de dado (Numérico, categórico ou text)
* O tamanho de cada coluna

Serão usados os seguintes pacotes:
* pandas
* numpy
* json
* savReaderWriter

Antes de executar este código faça o mapeamento das colunas para identificar os nomes e tipos. Se necessário ajuste o arquivo Excel.

In [1]:
import pandas as pd
import numpy as np
import json
import savReaderWriter as sav

In [2]:
# Definindo os nomes e caminhos dos arquivo
EXCEL_FILE = 'C:\Sandbox\dataset.xlsx'
JSON_FILE = 'C:/Sandbox/metadata.json'
SAV_FILE = 'C:\Sandbox\dataset.sav'

In [3]:
# ## Carregando e preparando os dados...
base = pd.read_excel(EXCEL_FILE)

# Tratar os dados missing
base.replace(to_replace=[None], value='', inplace=True) 
base.replace(to_replace=pd.np.nan, value='', inplace=True)

<ipython-input-3-ed935f35d1ea>:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  base.replace(to_replace=pd.np.nan, value='', inplace=True)


Definindo metadata do arquivo SPSS-SAV a partir do arquivo JSON
Este metadatata é formado pom um dicionário que tem a seguinte estrutura:
* Chave - coluna final do arquivo SPSS-SAV
* Valor - uma lista com: Nome Original, Var label, Var Type, Tamanho


In [4]:
with open(JSON_FILE) as f:
  metadata = json.load(f)


In [5]:
# ## Criando os dicionários e listas

renomar_colunas = {}
varNames = []
campos = []
varTypes = {}
varLabels = {}

for coluna in metadata:
    renomar_colunas[metadata[coluna][0]] = coluna
    if metadata[coluna][2] >0:
        varNames.append(coluna)
        
        if metadata[coluna][2] in [1,4]:
            campos.append(coluna)
        
        varTypes[coluna] = metadata[coluna][3]
        varLabels[coluna] = metadata[coluna][1]

In [6]:
# ## Renomeando as colunas do conjunto de dados

base.rename(columns=renomar_colunas, inplace = True)

In [7]:
# ## Transformando as colunas categóricas e capturando o *Value Labels* de cada coluna.

from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

valueLabels = {}

for campo in campos:
    base[campo] = lb_make.fit_transform(base[campo])
    itens = list(lb_make.classes_)

    valores = {}
    posicao = 0

    for item in itens:
        if item=='':
            valores[posicao] ='99999999'
        else:
            valores[posicao] =item

        posicao += 1

    valueLabels[campo] = valores

base_values = base[varNames].values


In [8]:
# ## Convertendo o dataframe
records = []
for case in base_values:
    record = []
    for value in case:
        if type(value) is str:
            record.append(value.encode())
        else:
            record.append(value)
    records.append(record)

In [9]:
# ## Salvando o arquivo SPSS-SAV

with sav.SavWriter(SAV_FILE, varNames, varTypes, varLabels=varLabels, valueLabels=valueLabels, 
                    ioUtf8=True,overwrite=True) as writer:
    for record in records:
        writer.writerow(record)

# Referências:
https://pbpython.com/categorical-encoding.html
 
https://pythonhosted.org/savReaderWriter/generated_api_documentation.html#savwriter
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

https://www.w3schools.com/python/python_dictionaries.asp
